In [1]:
import pandas as pd
import numpy as np
from inmoose.pycombat import pycombat_norm, pycombat_seq

In [116]:
#take the input from the directory
def batch_correction(filename,batch):
    df = pd.read_csv(filename,header = 1,sep = '\t')
    geneid_len = df[['Geneid','Length']]
    df = df.drop(['Chr','Start','End','Strand','Geneid','Length'], axis = 1 )
    for col in df.columns:
        df = df.rename(columns = {col : col.split('/')[-1].split('_')[0]})
    processed_data = pd.concat([geneid_len,df],axis = 1)
    
    categorical = pd.read_excel(batch)
    raw_read = processed_data.drop(['Geneid','Length'],axis = 1)
    processed_col = processed_data.columns[2:]
    processed_col = pd.DataFrame({'Sample_ID':processed_col})
    batch = processed_col.merge(categorical, on = 'Sample_ID',how = 'inner')
    batch_correct = np.array(batch['Batch'])
    correct_read = pycombat_seq(raw_read,batch_correct)
    correct_read = pd.concat([geneid_len,correct_read],axis=1)
    raw_read = pd.concat([geneid_len,raw_read],axis=1)
    raw_read = raw_read.transpose()
    correct_read = correct_read.transpose()
    raw_read = raw_read.reset_index()
    correct_read = correct_read.reset_index()
    meta_raw_read = raw_read['index']
    meta_correct_read = correct_read['index']
    raw_read = raw_read.rename(columns = {'index':'Sample_ID'} )
    correct_read = correct_read.rename(columns = {'index':'Sample_ID'} )
    raw_col_name = raw_read.iloc[0,1:]
    correct_col_name = correct_read.iloc[0,1:]
    raw_sample_id = raw_read['Sample_ID']
    correct_sample_id = correct_read['Sample_ID']
    raw_index = raw_read.iloc[0:,1:]
    correct_index = correct_read.iloc[0:,1:]
    raw_index.columns = raw_col_name
    correct_index.columns = correct_col_name
    raw_read = pd.concat([raw_sample_id,raw_index], axis = 1)
    correct_read = pd.concat([correct_sample_id,correct_index], axis = 1)
    raw_read = raw_read.iloc[1:]
    correct_read = correct_read.iloc[1:]

    
    
    
    
    return correct_read,raw_read
    

In [117]:
correct_read,raw_read = hope('read_count_documentaion_param.out','batch_category.xlsx')

[INFO] Found 5 batches
[INFO] Using null model
[INFO] Adjusting for 0 covariate(s) or covariate level(s)
[INFO] Estimating dispersions
[INFO] Fitting the GLM model
[INFO] shrinkage off - using GLM estimates for parameters
[INFO] Adjusting the data


In [118]:
correct_read.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
1,Length,570,626,9767,1225,3782,2407,8224,1697,337,...,1378,69,59,71,1812,525,69,1141,66,68
2,IGIB113000849750,4.0,4.0,4.0,2.0,52.0,5.0,30.0,0.0,0.0,...,2533.0,61.0,47.0,69.0,1552.0,510.0,152.0,2051.0,45.0,55.0


In [119]:
raw_read.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
1,Length,570,626,9767,1225,3782,2407,8224,1697,337,...,1378,69,59,71,1812,525,69,1141,66,68
2,IGIB113000849750,6,12,3,2,92,4,22,0,0,...,2755,93,69,84,1651,519,199,2296,63,69


In [211]:
def tpm_conversion(df):
    #get the dictionary of genelength and its name
    sample_id = df['Sample_ID']
    sample_id  = pd.DataFrame({'Sample_ID':sample_id[1:]}).reset_index().drop(['index'],axis =1 )
    gene_length = df.iloc[0,1:].values
    gene_name = np.array(df.columns[1:])
    gene_id_len_dict ={gene_id: float(length) for gene_id, length in zip(gene_name, gene_length)}
    data = df.drop(['Sample_ID'],axis =1 ).iloc[1:].reset_index().drop(['index'],axis =1)
    genes = np.array(df.columns[1:])
    data = np.array(data).astype(float)
    rpk = np.zeros(shape=(data.shape[0],data.shape[1]))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            gene_length = gene_id_len_dict[genes[j]]
            gene_length_in_kilo = gene_length/1000
            rpk[i,j] = data[i,j] / gene_length_in_kilo
    tpm = np.zeros(shape=(data.shape[0],data.shape[1]))

    for i in range(rpk.shape[0]):
        total_rpk = np.sum(rpk[i,:])
        scaled_total_rpk = total_rpk / 1000000
        for j in range(rpk.shape[1]):
            if scaled_total_rpk != 0:
                tpm[i,j] = rpk[i,j] / scaled_total_rpk
            else:
                tpm[i,j] = rpk[i,j]  # going to be 0

    
    tpm = pd.DataFrame(tpm)
    tpm.columns = gene_name
    df = pd.concat([sample_id,tpm],axis = 1)
    
    return df

In [346]:
correct_read_tpm = tpm_conversion(correct_read)

In [347]:
raw_read_tpm = tpm_conversion(raw_read)

In [349]:
def dseq_norm(df):
    tobeusedlaterforcol = df.columns[1:]
    df = df.iloc[1:,].reset_index().drop(['index'],axis =1)
    col = list(df['Sample_ID'])
    df  = df.iloc[:,1:].transpose().reset_index().drop(['index'],axis = 1)
    df.columns = col
    df = df.astype(float)
    df_log =  np.log(df)
    df_log['gene'] = df_log.index
    df_log['pseudo_reference'] = df_log.iloc[:, 0:-1].mean(axis=1)
    filtered_log_df = df_log[df_log['pseudo_reference'] != -float('inf')]
    ratio_data = filtered_log_df.iloc[:, 0:-2].subtract(filtered_log_df.iloc[:, -1], axis=0)
    sample_medians = ratio_data.median(axis=0)
    scaling_factors = np.exp(sample_medians)
    manually_normalized = df.divide(scaling_factors, axis=1)
    manually_normalized = manually_normalized.transpose()
    manually_normalized.columns = tobeusedlaterforcol
    manually_normalized = manually_normalized.reset_index()
    manually_normalized = manually_normalized.rename(columns = {'index':'Sample_ID'})
    return manually_normalized

In [350]:
correct_read_dseq = dseq_norm(correct_read)

[WARNING] /home/abuzarkhan_123/.local/lib/python3.10/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)



In [351]:
raw_read_dseq = dseq_norm(raw_read)

[WARNING] /home/abuzarkhan_123/.local/lib/python3.10/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)



In [358]:
correct_read.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
1,Length,570,626,9767,1225,3782,2407,8224,1697,337,...,1378,69,59,71,1812,525,69,1141,66,68
2,IGIB113000849750,4.0,4.0,4.0,2.0,52.0,5.0,30.0,0.0,0.0,...,2533.0,61.0,47.0,69.0,1552.0,510.0,152.0,2051.0,45.0,55.0


In [359]:
raw_read.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
1,Length,570,626,9767,1225,3782,2407,8224,1697,337,...,1378,69,59,71,1812,525,69,1141,66,68
2,IGIB113000849750,6,12,3,2,92,4,22,0,0,...,2755,93,69,84,1651,519,199,2296,63,69


In [353]:
raw_read_dseq.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
0,IGIB113000849750,1.205821,2.411642,0.602910,0.401940,18.489253,0.803881,4.421343,0.000000,0.000000,...,553.672726,18.690223,13.866939,16.881491,331.801695,104.303501,39.993057,461.427434,12.661119,13.866939
1,IGIB113001484596,18.217453,26.314099,60.724844,4.048323,20.241615,12.144969,38.459068,10.120807,10.120807,...,1692.198985,161.932917,305.648381,354.228256,1862.228548,862.292784,269.213475,1441.202963,105.256396,192.295339


In [354]:
raw_read.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
1,Length,570,626,9767,1225,3782,2407,8224,1697,337,...,1378,69,59,71,1812,525,69,1141,66,68
2,IGIB113000849750,6,12,3,2,92,4,22,0,0,...,2755,93,69,84,1651,519,199,2296,63,69


In [356]:
correct_read_dseq.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
0,IGIB113000849750,1.466765,1.466765,1.466765,0.733383,19.067948,1.833457,11.000739,0.00000,0.000000,...,928.829086,22.368170,17.234492,25.301700,569.104912,187.012568,55.737079,752.083875,16.501109,20.168022
1,IGIB113001484596,23.238986,40.668225,49.382845,5.809746,26.143859,17.429239,40.668225,8.71462,20.334113,...,1205.522389,104.575436,159.768028,174.292394,1225.856502,543.211294,174.292394,1089.327460,63.907211,107.480309


In [357]:
correct_read_tpm.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
0,IGIB113000849750,0.941437,0.857219,0.054942,0.219028,1.844539,0.278676,0.489378,0.000000,0.000000,...,246.599393,118.600570,106.869032,130.375736,114.905166,130.321750,295.529288,241.149323,91.469143,108.507513
1,IGIB113001484596,27.252384,43.425324,3.379692,3.170175,4.620727,4.840214,3.305478,3.432644,40.332719,...,584.774251,1013.077738,1810.089461,1640.900561,452.213600,691.626563,1688.462896,638.167944,647.244110,1056.530827


In [360]:
raw_read_tpm.head(2)

,Sample_ID,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,ENSG00000260972,ENSG00000224340,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
0,IGIB113000849750,1.109937,2.021291,0.032388,0.172154,2.565005,0.175229,0.282073,0.000000,0.000000,...,210.811525,142.120204,123.315891,124.750674,96.075100,104.238948,304.106673,212.181837,100.651112,106.994670
1,IGIB113001484596,20.915680,27.508909,4.068782,2.162710,3.502538,3.302018,3.060376,3.902946,19.653705,...,803.639731,1535.837389,3390.230881,3265.006422,672.564552,1074.866767,2553.329660,826.606258,1043.671317,1850.627589


## UMAP Plot TPM

In [373]:
def umap_tpm(raw,correct,batch):#these are normalized dataset:
    batch = pd.read_excel(batch)
    raw = batch.merge(raw,on = 'Sample_ID', how = 'left')
    correct = batch.merge(correct,on = 'Sample_ID', how = 'left')
    return correct
    
    
    

In [374]:
umap_tpm(raw_read_tpm,correct_read_tpm,'batch_category.xlsx')

,Sample_ID,Group,Batch,ENSG00000279928,ENSG00000228037,ENSG00000142611,ENSG00000284616,ENSG00000157911,ENSG00000269896,ENSG00000228463,...,ENSG00000198886,ENSG00000210176,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196
0,IGIB113083612,Mild,Batch2,1.511602,2.202206,0.052930,0.000000,2.095938,0.501146,0.146675,...,105.044245,104.892007,93.463096,55.822809,59.437977,95.187712,202.291728,99.527241,60.051809,55.751424
1,IGIB113002257842,Mild,Batch2,1.675425,1.144160,0.000000,0.000000,0.757529,0.694323,0.000000,...,169.618522,311.410477,323.726146,258.923923,138.874407,116.418088,401.373504,264.694357,72.347889,66.709009
2,IGIB113003405741,Mild,Batch4,1.142687,0.743190,0.076214,0.075957,0.762684,0.115971,0.045257,...,77.246890,59.334568,93.047390,70.768438,44.521020,40.054686,94.395903,82.119827,45.113886,61.575479
3,IGIB1130388376,Mild,Batch2,0.494307,4.950970,0.807735,0.920017,0.372495,0.351170,0.650942,...,242.497566,428.757886,840.490036,833.360399,235.728949,319.322540,706.429660,273.112385,239.065003,364.624354
4,IGIB1130493563,Mild,Batch5,3.377691,3.758985,0.000000,0.000000,2.149386,0.355497,0.416188,...,153.997866,406.138772,503.982011,337.451933,89.841855,77.826287,356.534037,166.111739,259.297481,276.838193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,IGIB113002297804,Severe,Batch3,1.095511,2.992529,0.079917,0.254874,1.527255,0.000000,0.170841,...,111.814826,196.834680,309.574590,288.034429,84.516649,114.183507,391.406891,139.418368,158.475570,202.025046
108,IGIB113002307585,Severe,Batch1,0.000000,4.062898,1.041619,2.076224,2.689978,1.056657,0.618525,...,2455.703252,4496.980081,6272.218884,5319.592058,1946.832070,2492.506501,7040.354062,3142.995015,2793.857782,3777.658412
109,IGIB113002393520,Severe,Batch1,15.508746,51.778401,3.620348,2.405438,3.116512,3.672615,4.657904,...,774.086759,2049.851653,3396.152527,3278.679977,655.355787,1077.636298,2690.430295,748.932435,1294.745315,1776.663703
110,IGIB1130489912,Severe,Batch5,3.421413,3.738412,0.000000,0.000000,3.506450,0.162044,0.047427,...,148.600539,203.499670,376.819304,368.066891,123.771299,157.502284,780.082067,221.855066,218.659367,309.738468


In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import umap

# Load your two dataframes (replace 'data1.csv' and 'data2.csv' with your actual data files)
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')

# Concatenate the two dataframes vertically for batch correction comparison
combined_df = pd.concat([df1, df2], ignore_index=True)

# Perform UMAP dimensionality reduction
reducer = umap.UMAP()
embedding = reducer.fit_transform(combined_df.iloc[:, 3:])  # Assuming gene columns start from column 3

# Add UMAP coordinates to the combined dataframe
combined_df['UMAP_1'] = embedding[:, 0]
combined_df['UMAP_2'] = embedding[:, 1]

# Create a UMAP plot using Plotly Express
fig = px.scatter(combined_df, x='UMAP_1', y='UMAP_2', color='Batch', symbol='Group',
                 hover_data=['Sample_ID'], title='UMAP Comparison with Batch Correction')

# Customize the plot appearance
fig.update_traces(marker=dict(size=8),
                  selector=dict(mode='markers'))

# Show the plot
fig.show()


In [377]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import umap
def umap_tpm(raw,correct,batch):#these are normalized dataset:
    batch = pd.read_excel(batch)
    raw = batch.merge(raw,on = 'Sample_ID', how = 'left')
    correct = batch.merge(correct,on = 'Sample_ID', how = 'left')
    reducer_correct = umap.UMAP()
    embedding_correct = reducer_correct.fit_transform(correct.iloc[:, 3:])
    correct['UMAP_1'] = embedding_correct[:, 0]
    correct['UMAP_2'] = embedding_correct[:, 1]
    reducer_raw = umap.UMAP()
    embedding_raw = reducer_raw.fit_transform(raw.iloc[:, 3:])
    raw['UMAP_1'] = embedding_raw[:, 0]
    raw['UMAP_2'] = embedding_raw[:, 1]
    
    fig = px.scatter(correct, x='UMAP_1', y='UMAP_2', color='Batch', symbol='Group',
                 hover_data=['Sample_ID'], title='UMAP Comparison with Batch Correction')
    fig.update_traces(marker=dict(size=8),
                  selector=dict(mode='markers'))
    
    
    
    
    
    return correct
    
    
    

ModuleNotFoundError: No module named 'plotly'

In [ ]:
umap_tpm(raw_read_tpm,correct_read_tpm,'batch_category.xlsx')

In [378]:
!pip install plotly

OSError: [Errno 12] Cannot allocate memory

In [384]:
correct_read_tpm.to_csv('correct_read_tpm.csv')


In [385]:
raw_read_tpm.to_csv('raw_read_tpm.csv')

In [386]:
correct_read_dseq.to_csv('correct_read_dseq.csv')

In [387]:
raw_read_dseq.to_csv('raw_read_dseq.csv')